In [46]:
!pip install requests
!pip install bs4
!pip install pandas

In [47]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def scrape_tradingeconomics(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Target the main stock market table (structure may change)
    table = soup.find("table", {"class": "table table-hover sortable-theme-minimal table-heatmap table-striped"})
    rows = table.find_all("tr")[1:]  # Skip header

    data = []
    for row in rows:
        cols = row.find_all("td")
        if len(cols) >= 4:
            # Extract components (e.g., MOEX Russia Index)
            component = cols[0].text.strip()
            price = cols[1].text.strip()

            # Append data (MCap may not be available on this page)
            data.append({
                "Company": component,
                "Price": price,
                "Day": cols[4].text.strip(),
                "Year": cols[5].text.strip(),
                "MCap": cols[6].text.strip(),
                "Date": cols[7].text.strip()
            })

    return pd.DataFrame(data)

# Example usage
url = "https://tradingeconomics.com/russia/stock-market"
try:
    df = scrape_tradingeconomics(url)
    print(df)
    df.to_json("data.json", orient="records", indent=4)
except Exception as e:
    print(f"Error: {e}. Check if the HTML structure changed.")

                               Company      Price     Day     Year     MCap  \
0                             Sberbank     320.95   1.06%    6.66%   80.15B   
1                              Rosneft     529.25   0.37%  -10.03%   65.17B   
2                               Lukoil   7,209.00   0.33%   -4.35%   54.76B   
3                              Gazprom     170.06  -0.60%    4.34%   47.07B   
4                              Novatek   1,263.20  -1.70%  -10.90%    45.3B   
..                                 ...        ...     ...      ...      ...   
60                   Akb Primor'ye Pao  34,800.00   1.16%   -3.33%  100.44M   
61                       Obyedinennaya       0.65  -0.15%  -54.08%    32.9M   
62                        Rusolovo Pao       0.81   1.61%  -34.66%   27.96M   
63                        Kazan Sintez      89.50  -1.32%  -17.36%   17.62M   
64  Rkk Energiya IM. S.P. Koroleva Pao  21,550.00  -0.23%  -10.10%     1.5M   

      Date  
0   Mar/11  
1   Mar/11  
2   Mar/11  